In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [5]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [7]:
# useful for getting number of output classes
folders = glob('/Users/macbookpro/Downloads/fruits/testing/fruits/*')

In [8]:
folders

['/Users/macbookpro/Downloads/fruits/testing/fruits/FreshBanana',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/RottenPomegranate',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/FreshJujube',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/FreshOrange',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/RottenOrange',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/RottenJujube',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/FreshGuava',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/RottenGrape',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/FreshApple',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/RottenBanana',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/RottenGuava',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/FreshStrawberry',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/RottenStrawberry',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/RottenApple',
 '/Users/macbookpro/Downloads/fruits/testing/fruits/FreshGra

In [9]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [10]:
len(folders)

16

In [11]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [12]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [14]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:

training_set = train_datagen.flow_from_directory('/Users/macbookpro/Downloads/fruits/testing/fruits',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 12335 images belonging to 16 classes.


In [16]:
test_set = test_datagen.flow_from_directory('/Users/macbookpro/Downloads/fruits/testing/fruits',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 12335 images belonging to 16 classes.


In [ ]:
# fit the model

r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
386/386 [==============================] - 5516s 14s/step - loss: 0.6387 - accuracy: 0.7969 - val_loss: 0.1781 - val_accuracy: 0.9454
Epoch 2/5
100/386 [======>.......................] - ETA: 30:19 - loss: 0.2515 - accuracy: 0.9205

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')